# Ola

A bit rough and ready for Dirk

Lets set some variables

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatools'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred

Lets check if Ola is already installed

In [2]:
Find-DbaStoredProcedure -SqlInstance $sqlinstances -SqlCredential $sqlCredential -Pattern CommandExecute  -IncludeSystemDatabases
Find-DbaStoredProcedure -SqlInstance $sqlinstances -SqlCredential $sqlCredential -Pattern DatabaseBackup  -IncludeSystemDatabases
Find-DbaStoredProcedure -SqlInstance $sqlinstances -SqlCredential $sqlCredential -Pattern DatabaseIntegrityCheck  -IncludeSystemDatabases
Find-DbaStoredProcedure -SqlInstance $sqlinstances -SqlCredential $sqlCredential -Pattern IndexOptimize  -IncludeSystemDatabases



ComputerName             : localhost
CreateDate               : 21/12/2019 14:33:52
Database                 : master
IsSystemObject           : False
LastModified             : 21/12/2019 14:35:23
Name                     : CommandExecute
Owner                    : dbo
Schema                   : dbo
SqlInstance              : MSSQLSERVER
StoredProcedureTextFound : (LineNumber: 1) CREATE PROCEDURE [dbo].[CommandExecute]

ComputerName             : localhost
CreateDate               : 21/12/2019 14:33:52
Database                 : master
IsSystemObject           : False
LastModified             : 21/12/2019 14:35:23
Name                     : DatabaseBackup
Owner                    : dbo
Schema                   : dbo
SqlInstance              : MSSQLSERVER
StoredProcedureTextFound : (LineNumber: 383) IF NOT EXISTS (SELECT * FROM sys.objects objects INNER JOIN sys.schemas 
                           schemas ON objects.[schema_id] = schemas.[schema_id] WHERE objects.[type] = 'P' AND 
  

Do we have the jobs ?

In [3]:
Get-DbaAgentJob  -SqlInstance $sqlinstances -SqlCredential $sqlCredential | ft


ComputerName InstanceName SqlInstance  Name                                     Category             OwnerLoginName Cur
                                                                                                                    ren
                                                                                                                    tRu
                                                                                                                    nSt
                                                                                                                    atu
                                                                                                                      s
------------ ------------ -----------  ----                                     --------             -------------- ---
localhost    MSSQLSERVER  50dd0f84a02b CommandLog Cleanup                       Database Maintenance OldSa          dle
localhost    MSSQLSERVER  50dd0f84a02b 

Get the schedule for one of them

In [5]:
Get-DbaAgentJob  -SqlInstance $sqlinstances -SqlCredential $sqlCredential -Job 'DatabaseBackup - SYSTEM_DATABASES - FULL'



ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : 50dd0f84a02b
Name                   : DatabaseBackup - SYSTEM_DATABASES - FULL
Category               : Database Maintenance
OwnerLoginName         : OldSa
CurrentRunStatus       : Idle
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 21/12/2019 14:41:56
LastRunOutcome         : Succeeded
HasSchedule            : False
OperatorToEmail        : 
CreateDate             : 21/12/2019 14:35:24

ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : edc37689b392
Name                   : DatabaseBackup - SYSTEM_DATABASES - FULL
Category               : Database Maintenance
OwnerLoginName         : OldSa
CurrentRunStatus       : Idle
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 21/12/2019 14:41:56
LastRunOutcome         : Succeeded
HasSchedule            : False
OperatorToEmail        :

No Schedule - HasSchedule = false - lets set a schedule for one of them

In [10]:
Get-DbaAgentSchedule -SqlInstance $SQLInstances -SqlCredential $sqlCredential |ft


ComputerName InstanceName SqlInstance  ScheduleName                      ActiveEndDate       ActiveEndTimeOfDay ActiveS
                                                                                                                tartDat
                                                                                                                e      
------------ ------------ -----------  ------------                      -------------       ------------------ -------
localhost    MSSQLSERVER  edc37689b392 CollectorSchedule_Every_10min     31/12/9999 00:00:00 23:59:59           24/0...
localhost    MSSQLSERVER  edc37689b392 CollectorSchedule_Every_15min     31/12/9999 00:00:00 23:59:59           24/0...
localhost    MSSQLSERVER  edc37689b392 CollectorSchedule_Every_30min     31/12/9999 00:00:00 23:59:59           24/0...
localhost    MSSQLSERVER  edc37689b392 CollectorSchedule_Every_5min      31/12/9999 00:00:00 23:59:59           24/0...
localhost    MSSQLSERVER  edc37689b392 

In [26]:
Set-DbaAgentJob -SqlInstance $SQLInstances -SqlCredential $sqlCredential -Job 'DatabaseBackup - SYSTEM_DATABASES - FULL' -Schedule CollectorSchedule_Every_6h



ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : 50dd0f84a02b
Name                   : DatabaseBackup - SYSTEM_DATABASES - FULL
Category               : Database Maintenance
OwnerLoginName         : OldSa
CurrentRunStatus       : Idle
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 01/01/0001 00:00:00
LastRunOutcome         : Unknown
HasSchedule            : False
OperatorToEmail        : 
CreateDate             : 26/03/2020 14:01:17

ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : edc37689b392
Name                   : DatabaseBackup - SYSTEM_DATABASES - FULL
Category               : Database Maintenance
OwnerLoginName         : OldSa
CurrentRunStatus       : Idle
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 01/01/0001 00:00:00
LastRunOutcome         : Unknown
HasSchedule            : False
OperatorToEmail        : 
Cr

So we have set a schedule for our jobs - Let's update maintenance solution and not lose the schedul

In [27]:
Install-DbaMaintenanceSolution -SqlInstance $sqlinstances -SqlCredential $sqlCredential -ReplaceExisting -InstallJobs:$False


ComputerName InstanceName SqlInstance     Results
------------ ------------ -----------     -------
localhost    MSSQLSERVER  localhost,15592 Success
localhost    MSSQLSERVER  localhost,15593 Success




In [28]:
Get-DbaAgentJob  -SqlInstance $sqlinstances -SqlCredential $sqlCredential -Job 'DatabaseBackup - SYSTEM_DATABASES - FULL'



ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : 50dd0f84a02b
Name                   : DatabaseBackup - SYSTEM_DATABASES - FULL
Category               : Database Maintenance
OwnerLoginName         : OldSa
CurrentRunStatus       : Idle
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 01/01/0001 00:00:00
LastRunOutcome         : Unknown
HasSchedule            : False
OperatorToEmail        : 
CreateDate             : 26/03/2020 14:01:17

ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : edc37689b392
Name                   : DatabaseBackup - SYSTEM_DATABASES - FULL
Category               : Database Maintenance
OwnerLoginName         : OldSa
CurrentRunStatus       : Idle
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 01/01/0001 00:00:00
LastRunOutcome         : Unknown
HasSchedule            : False
OperatorToEmail        : 
Cr

If we forget the $false parameter

In [24]:
Install-DbaMaintenanceSolution -SqlInstance $sqlinstances -SqlCredential $sqlCredential -ReplaceExisting -InstallJobs


ComputerName InstanceName SqlInstance     Results
------------ ------------ -----------     -------
localhost    MSSQLSERVER  localhost,15592 Success
localhost    MSSQLSERVER  localhost,15593 Success




In [25]:
Get-DbaAgentJob  -SqlInstance $sqlinstances -SqlCredential $sqlCredential -Job 'DatabaseBackup - SYSTEM_DATABASES - FULL'



ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : 50dd0f84a02b
Name                   : DatabaseBackup - SYSTEM_DATABASES - FULL
Category               : Database Maintenance
OwnerLoginName         : OldSa
CurrentRunStatus       : Idle
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 01/01/0001 00:00:00
LastRunOutcome         : Unknown
HasSchedule            : False
OperatorToEmail        : 
CreateDate             : 26/03/2020 14:01:17

ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : edc37689b392
Name                   : DatabaseBackup - SYSTEM_DATABASES - FULL
Category               : Database Maintenance
OwnerLoginName         : OldSa
CurrentRunStatus       : Idle
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 01/01/0001 00:00:00
LastRunOutcome         : Unknown
HasSchedule            : False
OperatorToEmail        : 
Cr